## Boilerplate

In [1]:
%run ../config/initialize.ipynb

import json
import shutil

## TODO
* how to generalize to regression problems?
  * easier to create a separate process?

## Set root dir path

In [2]:
ROOT='/Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/' \
     'models'
os.chdir(ROOT)

## Initialize
* set model ID
* remove this model ID's directory if it exists
* create directory

In [3]:
MODEL_ID = '0127_with_features_sklearn'

if os.path.exists(MODEL_ID):
    print 'Directory models/{} **EXISTS**'.format(MODEL_ID)    
else:
    print 'Directory models/{} **DOES NOT EXIST**'.format(MODEL_ID)    

Directory models/0127_with_features_sklearn **EXISTS**


In [4]:
## wipe out existing directory

if os.path.exists(MODEL_ID):
    shutil.rmtree(MODEL_ID)
os.mkdir(MODEL_ID)

## Generate dictionary version of model.json

In [5]:
model_dict = {'model_id': MODEL_ID}

### Source data for model

#### Hive Tables
* features
* tables

In [6]:
model_dict['features_tbl'] = 'features.combined_0127'

In [7]:
labels = spark.table('labels.team_game_line_labels').select(
        ## index
        'game_id','team_id',
        ## strata
        'is_home','is_fav_sbr',
        ## labels
        'did_win','final_margin','did_cover_pfr',
        'did_cover_sbr','did_cover_sbr_open'
    ).join(
        spark.table(model_dict['features_tbl']).select('game_id'), 
        on='game_id'
    )

labels.limit(5).toPandas()

,game_id,team_id,is_home,is_fav_sbr,did_win,final_margin,did_cover_pfr,did_cover_sbr,did_cover_sbr_open
0,201511150ram,chi,0.0,-1,1,24,1,1,1
1,201511090sdg,chi,0.0,-1,1,3,1,1,1
2,201510180det,chi,0.0,-1,-1,-3,0,1,0
3,201512270tam,chi,0.0,-1,1,5,1,1,1
4,201510110kan,chi,0.0,-1,1,1,1,1,1


In [8]:
labels_tbl = 'labels.0127_home_team'
labels.write.mode('overwrite').saveAsTable(labels_tbl)

In [9]:
model_dict['labels_tbl'] = labels_tbl

#### Columns from Hive tables
* index: unique identifier in features/labels table (must be in both)
* label column, and indicator of what is a positive label
  * currently not supported: multi-class
  * code will binarize
* list of features

In [10]:
model_dict['index'] = ['game_id']
model_dict['label_col'] = 'did_cover_sbr'
model_dict['pos_labels'] = [1]
model_dict['neg_labels'] = [-1]
model_dict['features_list'] = [
#  'time_date___game_time_eastern',
 'time_date___game_time_local',
 'time_date___day_of_week',
#  'time_date___is_sunday',
 'time_date___week_id',
#  'time_date___is_playoffs',
#  'time_date___is_superbowl',
#  'home_field___h_is_home',
#  'home_field___v_is_home',
#  'home_field___h_consecutive_home',
#  'home_field___v_consecutive_visitor',
#  'home_field___v_visitor__ovr_wr_ytd',
#  'home_field___v_visitor__open_wr_ytd',
#  'home_field___v_visitor__close_wr_ytd',
#  'home_field___v_visitor__pfr_wr_ytd',
#  'home_field___h_home__ovr_wr_ytd',
#  'home_field___h_home__open_wr_ytd',
#  'home_field___h_home__close_wr_ytd',
#  'home_field___h_home__pfr_wr_ytd',
#  'home_field___v_visitor__ovr_wr_last_4_games',
#  'home_field___v_visitor__ovr_wr_last_8_games',
#  'home_field___v_visitor__open_wr_last_4_games',
#  'home_field___v_visitor__open_wr_last_8_games',
#  'home_field___v_visitor__close_wr_last_4_games',
#  'home_field___v_visitor__close_wr_last_8_games',
#  'home_field___v_visitor__pfr_wr_last_4_games',
#  'home_field___v_visitor__pfr_wr_last_8_games',
#  'home_field___h_home__ovr_wr_last_4_games',
#  'home_field___h_home__ovr_wr_last_8_games',
#  'home_field___h_home__open_wr_last_4_games',
#  'home_field___h_home__open_wr_last_8_games',
#  'home_field___h_home__close_wr_last_4_games',
#  'home_field___h_home__close_wr_last_8_games',
#  'home_field___h_home__pfr_wr_last_4_games',
#  'home_field___h_home__pfr_wr_last_8_games',
#  'travel___v_dist_from_home',
#  'travel___v_home_body_time',
#  'travel___h_days_since_last_game',
#  'travel___h_tz_change_last_game',
#  'travel___h_tz_change_last_10_days',
#  'travel___h_travel_from_last_game',
#  'travel___h_travel_from_last_10_days',
#  'travel___h_travel_from_last_game_decay',
#  'travel___h_same_stadium_last_game',
#  'travel___h_body_clock_from_last_wk',
#  'travel___v_days_since_last_game',
#  'travel___v_tz_change_last_game',
#  'travel___v_tz_change_last_10_days',
#  'travel___v_travel_from_last_game',
#  'travel___v_travel_from_last_10_days',
#  'travel___v_travel_from_last_game_decay',
#  'travel___v_same_stadium_last_game',
#  'travel___v_body_clock_from_last_wk',
#  'weather___wind_chill',
#  'weather___humidity_pct',
#  'weather___temperature',
#  'weather___wind_mph',
#  'weather___is_grass',
#  'weather___is_dome',
#  'matchup___is_same_division',
#  'matchup___is_same_conf',
#  'matchup___h_wr_past_5_games',
#  'matchup___h_wr_ytd',
#  'matchup___h_wr_past_2_seasons',
#  'matchup___h_wr_past_3_seasons',
 'line___h_ml',
#  'line___v_ml',
#  'line___open_ou',
#  'line___pfr_ou',
 'line___close_ou',
#  'line___h_open_line',
#  'line___h_pfr_line',
 'line___h_close_line',
#  'team_history___h_open_wr_past_5_games',
#  'team_history___h_close_wr_past_5_games',
#  'team_history___h_pfr_wr_past_5_games',
#  'team_history___h_ovr_wr_past_5_games',
#  'team_history___h_open_wr_ytd',
#  'team_history___h_close_wr_ytd',
#  'team_history___h_pfr_wr_ytd',
#  'team_history___h_ovr_wr_ytd',
#  'team_history___h_open_wr_past_2_seasons',
#  'team_history___h_close_wr_past_2_seasons',
#  'team_history___h_pfr_wr_past_2_seasons',
#  'team_history___h_ovr_wr_past_2_seasons',
#  'team_history___h_open_wr_past_3_seasons',
#  'team_history___h_close_wr_past_3_seasons',
#  'team_history___h_pfr_wr_past_3_seasons',
#  'team_history___h_ovr_wr_past_3_seasons',
#  'team_history___v_open_wr_past_5_games',
#  'team_history___v_close_wr_past_5_games',
#  'team_history___v_pfr_wr_past_5_games',
#  'team_history___v_ovr_wr_past_5_games',
#  'team_history___v_open_wr_ytd',
#  'team_history___v_close_wr_ytd',
#  'team_history___v_pfr_wr_ytd',
#  'team_history___v_ovr_wr_ytd',
#  'team_history___v_open_wr_past_2_seasons',
#  'team_history___v_close_wr_past_2_seasons',
#  'team_history___v_pfr_wr_past_2_seasons',
#  'team_history___v_ovr_wr_past_2_seasons',
#  'team_history___v_open_wr_past_3_seasons',
#  'team_history___v_close_wr_past_3_seasons',
#  'team_history___v_pfr_wr_past_3_seasons',
#  'team_history___v_ovr_wr_past_3_seasons'
]

model_dict['features_list'].sort()

In [11]:
model_dict['features_list']

['line___close_ou',
 'line___h_close_line',
 'line___h_ml',
 'time_date___day_of_week',
 'time_date___game_time_local',
 'time_date___week_id']

In [12]:
# ['home_field___h_consecutive_home', 'home_field___h_home__close_wr_last_8_games', 'home_field___h_home__close_wr_ytd', 'home_field___h_home__open_wr_last_8_games', 'home_field___h_home__ovr_wr_last_8_games', 'home_field___h_home__pfr_wr_last_8_games', 'home_field___v_consecutive_visitor', 'home_field___v_visitor__close_wr_last_8_games', 'home_field___v_visitor__close_wr_ytd', 'home_field___v_visitor__open_wr_last_8_games', 'home_field___v_visitor__ovr_wr_last_8_games', 'home_field___v_visitor__ovr_wr_ytd', 'home_field___v_visitor__pfr_wr_last_8_games', 'line___close_ou', 'line___h_close_line', 'line___h_ml', 'matchup___h_wr_past_3_seasons', 'matchup___h_wr_past_5_games', 'matchup___h_wr_ytd', 'matchup___is_same_division', 'team_history___h_close_wr_past_2_seasons', 'team_history___h_close_wr_past_3_seasons', 'team_history___h_close_wr_past_5_games', 'team_history___h_close_wr_ytd', 'team_history___v_close_wr_past_2_seasons', 'team_history___v_close_wr_past_3_seasons', 'team_history___v_close_wr_past_5_games', 'team_history___v_close_wr_ytd', 'time_date___day_of_week', 'time_date___game_time_local', 'time_date___is_playoffs', 'time_date___week_id', 'travel___h_body_clock_from_last_wk', 'travel___h_days_since_last_game', 'travel___h_travel_from_last_game_decay', 'travel___h_tz_change_last_10_days', 'travel___v_body_clock_from_last_wk', 'travel___v_days_since_last_game', 'travel___v_home_body_time', 'travel___v_travel_from_last_game_decay', 'travel___v_tz_change_last_10_days', 'weather___humidity_pct', 'weather___is_dome', 'weather___is_grass', 'weather___temperature', 'weather___wind_chill', 'weather___wind_mph']

#### Checks

In [13]:
## assert these fields are of the correct type
assert type(model_dict['index']) is list
assert type(model_dict['label_col']) is str
assert type(model_dict['features_tbl']) is str
assert type(model_dict['features_tbl']) is str
assert type(model_dict['features_list']) is list
assert type(model_dict['pos_labels']) is list

## assert format is schema.table and that
## table exists in hive
for tbl_str in ['features_tbl','features_tbl']:
    schema_and_tbl = model_dict[tbl_str].split('.')
    assert len(schema_and_tbl) == 2
    schema, tbl = schema_and_tbl
    assert spark.sql(
            'show tables in {}'.format(schema)
        ).filter(
            col('tableName') == tbl
        ).count() == 1

feat_cols_set = set(spark.table(model_dict['features_tbl']).columns)
label_cols_set = set(spark.table(model_dict['labels_tbl']).columns)
idx_set = set(model_dict['index'])
feat_set = set(model_dict['features_list'])
label_set = set([model_dict['label_col']])

## assert the chosen columns exist in the
## chosen tables
assert not idx_set - feat_cols_set
assert not idx_set - label_cols_set
assert not feat_set - feat_cols_set
assert not label_set - label_cols_set

## check that positive and negative label values 
## are valid
for label_val in ['pos_labels','neg_labels']:
    assert spark.table(
            model_dict['labels_tbl']
        ).filter(
            col(model_dict['label_col']).isin(model_dict[label_val])
        ).count() > 0

### Cross-Validation Sets
* random seeds for reproducibility
* number of folds for cross-validation (value of <= 1 doesn't do k-fold
* __TODO__: ability to call another's CV set
* global_dataset_proportions
 * proportion of the data for each of training, scoring only, holdout, and throwaway
 * generated using stratified sampling
* dimensional_dataset_proportions
 * post-processing after global_dataset_proportions
 * idea is to move specific field values, e.g. move certain seasons to the holdout set

#### Optional: use cross-validation data from another model

In [14]:
model_dict['model_cv_to_use'] = None

#### CV parameters, when not using another model CV

In [15]:
model_dict['kfold_seed'] = 99
model_dict['dataset_seed'] = 9
model_dict['kfolds'] = 5
model_dict['strata_cols'] = ['did_cover_sbr']
model_dict['holdout_set'] = {
    'store_to_disk': False,
    'score_using_full_model': False 
}

model_dict['global_dataset_proportions'] = {
        'in_training': 1.,
        'holdout': 0,
        'throw_away': 0,
        'scoring_only': 0
    }

## SAMPLE USAGE:
_ = '''
model_dict['dimensional_dataset_proportions'] = {
        'throw_away': [
            {
                'vals': [
                    0
                ], 
                'dim': 'is_home',
                'prop_to_move': 1.0, 
                'from_groups': [
                    'in_training',
                    'holdout',
                    'scoring_only'
                ]
            }
        ]
    }
'''
model_dict['dimensional_dataset_proportions'] = {}

#### Checks

In [16]:
if model_dict['model_cv_to_use']:
    assert type(model_dict['model_cv_to_use']) in [str, unicode]
    assert os.path.exists(model_dict['model_cv_to_use'])
else:
    ## assert the data structures/types are correct
    assert type(model_dict['kfold_seed']) is int
    assert type(model_dict['dataset_seed']) is int
    assert type(model_dict['kfolds']) is int
    assert type(model_dict['strata_cols']) is list
    assert type(model_dict['global_dataset_proportions']) is dict
    assert type(model_dict['dimensional_dataset_proportions']) is dict
    assert type(model_dict['holdout_set']) is dict

    ## assert strata cols are present in the labels table
    assert not set(model_dict['strata_cols']) - label_cols_set

    dataset_types = set(['in_training','holdout','throw_away','scoring_only'])
    global_datasets = model_dict['global_dataset_proportions']
    dim_datasets = model_dict['dimensional_dataset_proportions']

    ## assert global_dataset_proportions has all possible dataset types
    assert set(global_datasets.keys()) == dataset_types
    ## values are proportions that must sum to 1
    assert sum(global_datasets.values()) == 1
    ## assert that the keys are valid dataset types
    assert not set(dim_datasets.keys()) - dataset_types
    ## assert the following (in order of assertion block):
    ## (1) each value is a list
    ## (2) each element of the list is a dict
    ## (3) each dict has the 5 required keys
    ## (4) the "dim" field is in the strata columns 
    ## (5) "prop_to_move" field is [0, 1]
    ## (6) "from_groups" are in the possible dataset types
    for k, dim_list in dim_datasets.iteritems():
        assert (type(dim_list)) is list
        for entry in dim_list:
            assert type(entry) is dict
            assert set(entry.keys()) \
                    == set(['vals','dim','prop_to_move','from_groups'])
            assert entry['dim'] in model_dict['strata_cols']
            assert 0 <= entry['prop_to_move'] <= 1
            assert not set(entry['from_groups']) - dataset_types

    ## assert holdout set has 2 keys (store_to_disk, score_using_full_model)
    ## and the corresponding values are boolean
    assert set(model_dict['holdout_set'].keys()) \
            == set(['store_to_disk','score_using_full_model'])
    assert len(filter(
        lambda x: type(x) is not bool, 
        model_dict['holdout_set'].values()
    )) == 0
    ## if holdout data isn't stored, it can't be scored
    assert not (model_dict['holdout_set']['store_to_disk'] is False) \
                & (model_dict['holdout_set']['score_using_full_model'] is True)

### Model Choice
* package/class name as a string
* parameters as a dictionary

#### sklearn basic

In [17]:
# model_dict['model'] = 'sklearn.ensemble.GradientBoostingClassifier'
# model_dict['model_params'] = {
#     'learning_rate': 0.1, 
#     'n_estimators': 200, 
#     'max_features': 'auto', 
#     'subsample': 0.9, 
#     'random_state': 9, 
#     'max_depth': 12, 
# }

#### xgboost basic

In [18]:
model_dict['model'] = 'xgboost.XGBClassifier'
model_dict['model_params'] = {
        'n_jobs': 1, 
        'learning_rate': 0.1, 
        'n_estimators': 200, 
        'max_features': 'auto', 
        'booster': 'gbtree', 
        'silent': True, 
        'nthread': None, 
        'subsample': 0.9, 
        'random_state': 9, 
        'objective': 'binary:logistic', 
        'max_depth': 12, 
        'gamma': 0
    }

#### Checks

In [19]:
## test that model object can be created
## from model inputs
try:
    import importlib

    model_class_str = model_dict['model']
    model_obj_path = '.'.join(model_class_str.split('.')[:-1])
    model_name = model_class_str.split('.')[-1]
    model_package = importlib.import_module(model_obj_path)
    model_class = getattr(model_package, model_name)
    _ = model_class(**model_dict['model_params'])
except Exception as e:
    e

### Write out model.json file

In [20]:
model_json_path = '{}/model.json'.format(model_dict['model_id'])
assert not os.path.exists(model_json_path)

with open(model_json_path,'w') as w:
    json.dump(model_dict, w, indent=4)

## Generate dictionary version of plots.json

In [21]:
plots_dict = {'model_id': MODEL_ID}

### Plot Labels
* labels --> names (note: keys should be strings)
* name for success rate

In [22]:
plots_dict['label_map'] = {
    '1': 'Covered',
    '0': 'Did Not Cover'
}
plots_dict['success_name'] = 'Cover Rate'

#### Checks

In [23]:
assert type(plots_dict['label_map']) is dict
assert type(plots_dict['success_name']) is str
assert set(plots_dict['label_map'].keys()) == set(['0','1'])

### Bins to plot
* plot_bins: 
   * Number of bins to plot (i.e. number of bars on the bar chart)
* bin_types:
   * "Bin" puts scores into uniform bins, e.g. [0, 0.10], (0.10, 0.20], ..., (0.9, 1.0]
   * "Percentile" bins scores into ntiles determined by plot_bins

In [24]:
plots_dict['bin_types'] = ['Bin', 'Percentile']
plots_dict['plot_bins'] = [10, 100]

#### Checks

In [25]:
## currently only supports "Bin" and "Percentile"
assert not set(plots_dict['bin_types']) - set(['Bin','Percentile'])
## all plot bins values should be ints
assert plots_dict['plot_bins'] == map(int, plots_dict['plot_bins'])
## ensure all bins values are in [2, 1000]
assert filter(
        lambda x: 2 <= x <= 1000, plots_dict['plot_bins']
    )   == plots_dict['plot_bins']

### Threshold Metrics to Plot
* metrics evaluated at each of 100 score threshold points
* currently only supports Accuracy and F1

In [26]:
plots_dict['threshold_metrics'] = ['Accuracy','F1']

#### Checks

In [27]:
assert type(plots_dict['threshold_metrics']) is list
## currently only supports Accuracy and F1
assert not set(plots_dict['threshold_metrics']) - set(['Accuracy','F1'])

### Write out plots.json

In [28]:
plots_json_path = '{}/plots.json'.format(model_dict['model_id'])
assert not os.path.exists(plots_json_path)

with open(plots_json_path,'w') as w:
    json.dump(plots_dict, w, indent=4)

## Execute pipeline

In [32]:
MODEL_ID
!source ~/.bashrc && \
    unset PYSPARK_PYTHON && \
    unset PYSPARK_DRIVER_PYTHON && \
    unset PYSPARK_DRIVER_PYTHON_OPTS && \
    cd /Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/ && \
    model_pipeline/model_pipeline.sh {MODEL_ID}

'0127_with_features_sklearn'

Check JSON files

real	0m13.124s
user	0m26.397s
sys	0m2.229s

Cross-validation data

real	0m19.593s
user	1m1.179s
sys	0m4.291s

Train and score

real	0m13.472s
user	0m12.257s
sys	0m1.097s

Evaluate and plot

real	0m8.361s
user	0m7.506s
sys	0m1.168s


In [30]:
!cat /Users/joshplotkin/Dropbox/data_science/modeling-football-outcomes/models/{MODEL_ID}/logs/out

JSON configuration files passed checks.
cv sets wrote successfully.
successfully completed evaluation and plotting.


In [31]:
!open {MODEL_ID}